In [76]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
from lightgbm import LGBMRegressor
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV, train_test_split
from pykrx import stock
from pykrx import bond
from pandas import DataFrame, Series
from io import BytesIO
from bs4 import BeautifulSoup
from sklearn.feature_selection import RFE, RFECV, SelectFromModel
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, StratifiedKFold, KFold
from sklearn.linear_model import LogisticRegression, Lasso, LassoCV
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score
import platform
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import  train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

rfc = RandomForestRegressor()
xgb = XGBRegressor()
lr = LinearRegression()
lgbm = LGBMRegressor()
svc = LinearSVC()

In [84]:
y = pd.read_csv('./y_open_0905~_1000(5min)/y_open_0930.csv', index_col=0)
X = pd.read_csv('./RETURN.csv', index_col=0)

from datetime import datetime


X['date'] = pd.to_datetime(X['date'] ,format="%Y-%m-%d")
y['date'] = pd.to_datetime(y['date'],format= "%Y%m%d")
df = pd.merge(X,y, on='date',how='inner')
df['open']=df['open'].shift(1)
df.set_index('date', inplace=True)
df.dropna(inplace=True)
y = df[['open']]
X = df.drop(columns='open')

In [85]:
X

,GOLD_RETURN,MEAL_RETURN,SP_500_RETURN,USD_KRW_RETURN,VIX_RETURN,WTI_RETURN,BTC_RETURN,KR_3Y,KR_10Y,USA_3Y,...,kodex_low,kodex_close,kodex_volume,kosdaq_open,kosdaq_high,kosdaq_low,kosdaq_close,kosdaq_volume,KODEX_return,Kosdaq150_return
date,,,,,,,,,,,,,,,,,,,,,
2023-10-25,13.5,-11.50,-46,-9.0,0.80,1.70,604000.0,4.03,4.259,4.979,...,31830,31840,3758074,12495,12510,12060,12060,11663976,-485.0,-345.0
2023-10-24,1.7,-7.25,12,7.0,-1.06,-2.41,190000.0,4.01,4.273,4.910,...,31375,32015,5387775,12215,12470,11885,12455,24326802,90.0,40.0
2023-10-23,0.3,0.75,7,15.0,-1.46,-2.51,2997000.0,4.05,4.366,4.864,...,31715,31770,3815317,12095,12250,12040,12090,17232920,150.0,125.0
2023-10-20,6.4,-8.00,-49,6.0,0.12,-1.53,588000.0,4.04,4.332,4.913,...,31815,31940,5747441,12205,12335,12000,12130,22987290,-35.0,-35.0
2023-10-19,20.4,14.25,-43,-3.0,1.67,1.13,255000.0,4.08,4.378,5.018,...,32410,32430,5179844,12480,12525,12345,12385,13563027,-255.0,-180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-10-08,-18.1,-10.00,7,-1.0,-0.36,-0.11,73000.0,2.09,2.455,2.968,...,26176,26185,5286714,12661,12895,12508,12508,6036573,45.0,9.0
2018-10-05,1.9,3.00,-17,-7.0,0.53,-0.33,70000.0,2.09,2.455,2.985,...,26140,26316,8808307,12881,13020,12603,12670,4261503,-122.0,-9.0
2018-10-04,0.2,2.25,-18,-14.0,1.38,-1.85,45000.0,2.07,2.446,2.964,...,26307,26356,8133088,13063,13130,12799,12924,4506692,-40.0,-43.0


In [86]:
# 모델을 통한 피처셀렉
def feature_select(X, y, min_features = 10, step = 1, cv = 5) :
    model = RandomForestRegressor(random_state=17, n_estimators=100)
    selector = RFECV(model, step=step, cv=cv, min_features_to_select=min_features)
    selector = selector.fit(X, y)
    selected_features = X.columns[selector.support_]
    return selected_features

selected_features = feature_select(X, y, cv = 5)

c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConvers

In [44]:
selected_features

Index(['BTC_RETURN', 'KR_3Y', 'KR_10Y', 'kodex_open', 'kodex_high',
       'kodex_low', 'kodex_close', 'kosdaq_open', 'kosdaq_high', 'kosdaq_low'],
      dtype='object')

In [87]:
X = X[['BTC_RETURN', 'KR_3Y', 'KR_10Y', 'kodex_open', 'kodex_high','kodex_low', 'kodex_close', 'kosdaq_open', 'kosdaq_high', 'kosdaq_low']]

In [88]:
X_train, y_train, X_test, y_test = train_test_split(X,y,test_size=0.2,shuffle=False)

In [89]:
X_test

,open
date,
2023-10-25,31355.0
2023-10-24,32095.0
2023-10-23,31810.0
2023-10-20,31845.0
2023-10-19,31980.0
...,...
2019-10-15,25569.0
2019-10-14,25326.0
2019-10-11,25362.0


In [73]:
def rf_model(X_train, y_train, X_test, y_test, cv=5) :

    # 최적의 하이퍼 파라미터 찾기
    param_grid = {
        'n_estimators': [50, 100, 150],
        'max_depth': [None, 5, 10, 15],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    }

    # rf 초기화
    rf = RandomForestRegressor(random_state=42)

    # Grid search
    grid_search = GridSearchCV(rf, param_grid, cv=cv, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, X_test)

    # 최적값을 모델에 적용
    best_params = grid_search.best_params_
    optimized_rf = RandomForestRegressor(**best_params, random_state=42)

    # Train the model using the training data
    optimized_rf.fit(X_train, X_test)

    # Predict on the test set
    y_pred_rf = optimized_rf.predict(y_train)

    # Calculate r2_score
    rf_r2_score = r2_score(y_test, y_pred_rf)

    y_pred_rf = pd.DataFrame(y_pred_rf, index=y_test.index, columns=['signal'])
    print(best_params, rf_r2_score)
    return y_pred_rf

In [74]:
rf_model(X_train, y_train, X_test, y_test, cv=5)

c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
c:\Users\hotte\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1151: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


{'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100} 0.9855005518207713


,signal
date,
2019-10-07,24662.879833
2019-10-04,24571.818167
2019-10-02,24962.180000
2019-10-01,25244.204905
2019-09-30,25036.818333
...,...
2018-10-08,26208.727966
2018-10-05,26351.490816
2018-10-04,26696.164000


In [ ]:
y

In [57]:
def xgb_model(X_train, y_train, X_test, y_test, cv=5) :

    # 최적의 하이퍼 파라미터 찾기
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1, 0.5],
        'n_estimators': [50, 100, 150, 300,],
        'max_depth': [3, 5, 7],
        'gamma': [0, 0.1, 0.2],
        'subsample': [0.8, 1],
        'colsample_bytree': [0.8, 1]
    }


    # xgb 초기화
    xgb = XGBRegressor(random_state=42)

    # Grid search
    grid_search = GridSearchCV(rf, param_grid, cv=cv, scoring='r2', n_jobs=-1)
    grid_search.fit(X_train, X_test)

    # 최적값을 모델에 적용
    best_params = grid_search.best_params_
    optimized_rf = XGBRegressor(**best_params, random_state=42)

    # Train the model using the training data
    optimized_rf.fit(X_train, X_test)

    # Predict on the test set
    y_pred_rf = optimized_rf.predict(y_train)

    # Calculate r2_score
    xgb_r2_score = r2_score(y_test, y_pred_rf)

    y_pred_rf = pd.DataFrame(y_pred_rf, index=y_test.index, columns=['signal'])
    print(best_params, rf_r2_score)
    return y_pred_rf

ValueError: Found input variables with inconsistent numbers of samples: [960, 240]